# 1초 단위로 뽑았을 때 피크를 더 디텍트 잘하는것같음
# 파일 1개 기준(약30초 ) 이면 1초 단위로 뽑아서 피크 인덱스 디텍트 하여서 
# 인덱스 구해서 extend  하는 형식으로 정확한 피크점 잡아볼 예정

In [17]:
import tensorflow as tf

In [18]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display
import math
import sys
#sys.path.insert(0,'/home/ikwak2/hmd/notebooks')
sys.path.insert(0,'lucashnegri-peakutils-51a679cd8428')
# sys.path.insert(0,'S1-S1-Phonocardiogram-Peak-Detection-Method-in-Python')
sys.path.insert(0,'utils')
from helper_code import *
from get_feature import *
# from models import *
from Generator0 import *
from keras.preprocessing import sequence
# import peakutils
from scipy import special
import scipy.io as sio
from keras.preprocessing.sequence import pad_sequences
import peakutils
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint

In [19]:
train_folder =  '/home/jk21/Downloads/Data/data/murmur/train'
test_folder = '/home/jk21/Downloads/Data/data/murmur/test'

In [20]:
patient_files_trn = find_patient_files(train_folder)
patient_files_test = find_patient_files(test_folder)

In [21]:
data_folder = train_folder

In [22]:
num_patient_files = len(patient_files_trn)
current_patient_data = load_patient_data(patient_files_trn[369])
num_locations = get_num_locations(current_patient_data)
recording_information = current_patient_data.split('\n')[1:num_locations+1]
entries = recording_information[1].split(' ')
recording_file = entries[2]
filename = os.path.join(data_folder, recording_file)

In [23]:
filename

'/home/jk21/Downloads/Data/data/murmur/train/68394_MV.wav'

In [24]:
                datos=sp.io.wavfile.read(filename)
                filtros=sio.loadmat('./Filters1')

In [25]:
per_sec = 16000

In [26]:
import matplotlib.pyplot as plt                 # Librery to load .mat files
import peakutils
from scipy import special
import scipy.io as sio
from scipy import signal

In [27]:
test_interval = []

for i in tqdm.tqdm(range(len(patient_files_trn))):
    current_patient_data = load_patient_data(patient_files_trn[i])
    num_locations = get_num_locations(current_patient_data)
    recording_information = current_patient_data.split('\n')[1:num_locations+1]
    
    for j in range(num_locations) :
        entries = recording_information[j].split(' ')
        recording_file = entries[2]
        filename = os.path.join(data_folder, recording_file)
        
        
        datos=sp.io.wavfile.read(filename)
        filtros=sio.loadmat('/home/jk21/Documents/hmd/jk_classifier/S1-S1-Phonocardiogram-Peak-Detection-Method-in-Python/Filters1')
        tmp_interval = []
        n_samp = len(datos[1])//per_sec
        
        for k in range(n_samp):
            X = datos[1][k*per_sec:(k+1)*per_sec]
            Fs= datos[0]
            Fpa20=filtros['Fpa20'];			        # High pass filter
            Fpa20=Fpa20[0];					# High pass filter
            Fpb100=filtros['Fpb100'];		        # Low-pass Filter
            Fpb100=Fpb100[0];				# Low-pass Filter
            Xf=FpassBand(X,Fpa20,Fpb100); 	                # Apply a passband filter
            Xf=vec_nor(Xf);			
            
            # Derivate of the Signal
            dX=derivate(Xf);				# Derivate of the signal
            dX=vec_nor(dX);					# Vector Normalizing
            # Square of the signal
            dy=np.square(Xf);
            dy=vec_nor(dy);
                    
            size=np.shape(Xf)				# Rank or dimension of the array
            fil=size[0];					# Number of rows

            positive=np.zeros((1,fil+1));                   # Initializating Positives Values Vector 
            positive=positive[0];                           # Getting the Vector

            points=np.zeros((1,fil));                       # Initializating the all Peak Points Vector
            points=points[0];                               # Getting the point vector

            peaks=np.zeros((1,fil));                        # Initializating the s1-s1 Peak Vector
            peaks=peaks[0];                                 # Getting the point vector

            
            for i in range(0,fil):
                if dX[i]>0:
                    positive[i]=1;
                else:
                    positive[i]=0;
    
            for i in range(0,fil):
                if (positive[i]==1 and positive[i+1]==0):
                    points[i]=Xf[i];
                else:
                    points[i]=0;

            indexes=peakutils.indexes(points,thres=0.5/max(points), min_dist=500);
            lenght=np.shape(indexes)			# Get the length of the index vector		
            lenght=lenght[0];				# Get the value of the index vector

            for i in range(0,lenght):
                p=indexes[i];
                peaks[p]=points[p];
        
            n=np.arange(0,fil);                            # Vector to the X axes (Number of Samples)
            
#             plt.figure(0)
#             plt.subplot(3,1,1)
#             plt.plot(n,Xf)
#             plt.grid(True)
#             plt.title('Normal Phonocardiogram Signal')
#             plt.subplot(3,1,2)
#             plt.plot(n,Xf,'k',n,points,'bo')
#             plt.grid(True)
#             plt.title('Phonocardiogram Signal All Peaks Points')
#             plt.ylabel('Normalized Amplitude',fontsize=12)
#             plt.subplot(3,1,3)
#             plt.plot(n,Xf,'k',n,peaks,'bo')
#             plt.title('Phonocardiogram Signal s1-s1 Peak Detection')
#             plt.xlabel('Number of Samples',fontsize=12)
#             plt.grid(True)


#             plt.figure(1)
#             plt.plot(n,Xf,'k',n,peaks,'bo')
#             plt.ylabel('Normalized Amplitude',fontsize=12)
#             plt.xlabel('Number of Samples',fontsize=12)
#             plt.title('Phonocardiogram Signal s1-s1 Peak Detection')
#             plt.grid(True)
#             plt.show()

            
            
            indexes =indexes+(k*per_sec)    
            tmp_peaks = np.array(indexes)
                    
                    
            tmp_interval.extend(tmp_peaks)
        
        tmp_interval = np.array(tmp_interval)
        tmp_interval = np.diff(tmp_interval)
        test_interval.append(tmp_interval)

test_interval = np.array(test_interval)

100%|█████████████████████████████████████████| 751/751 [06:46<00:00,  1.85it/s]
/tmp/ipykernel_60564/2821099257.py:108: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_interval = np.array(test_interval)


In [28]:
test_interval

array([array([ 1185,   887,  1195,  1084,  1105,  1089,  1144,   870,  1092,
                929,  1067,  1070,  1104,  1071,   855,   551,  1100,  1057,
                981,  1355,   892,  1240,  1111,  1259,   990,  1183,  1100,
                644,   581,   973,  1209,   893,  1395,  1006,  1239,  1104,
               1334,  1120,  1187,  1284,   786,   884,  1067,  1110,  1037,
               2921,   505, 17409,  7721,  3104])                           ,
       array([ 577,  683, 1000, 1291,  991, 1187,  978,  952, 1178, 1157, 1018,
              1183, 1017, 1250, 1009, 1181,  986,  763, 1255, 1049, 1387,  624,
               993, 1042,  988, 1128,  978, 2089, 1189,  832, 1025, 2128, 1008,
              1696, 2619, 1017, 1440, 1004, 1366, 1012, 1319])                 ,
       array([1005, 1172,  863, 1175,  960,  970, 1007, 1214,  861, 1147,  974,
               564, 1511, 1171,  826, 1168,  849, 1197,  874, 1079,  997, 1183,
               998, 1352,  858, 1240,  850,  639,  567, 

In [29]:
total_length=[]
for i in range(len(test_interval)):
    tmp_length=len(test_interval[i])
    total_length.append(tmp_length)

In [30]:
np.max(np.array(total_length))

192